# Unit Model Configuration

When undertaking a modeling exercise for a particular process, there can be any number of different approaches, assumptions, or relevant phenomena to include (or not include) in the model that is used. In WaterTAP, these discrete decisions are made at the point of model instantiation via passing of _configuration arguments_.

Passing different configuration arguments will typically result in the creation (or exclusion) of specific variables and constraints.

Some models have many configuration arguments and some have zero. All WaterTAP unit process models require passing the required property model via the `property_package` configuration argument. This tutorial will go through configuration arguments for the WaterTAP pump and RO model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    value,
    assert_optimal_termination,
    units as pyunits,
)

# Imports from IDAES
from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor

# Imports from WaterTAP
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump
from watertap.unit_models.reverse_osmosis_0D import ReverseOsmosis0D
from watertap.core.solvers import get_solver

solver = get_solver()

# Pump Configuration Arguments

Every WaterTAP model has default values for every configuration argument (except `property_package`). Using different model configurations will create additional variables and constraints that require additional parameter data from the user. 

The example below presents two different builds for the WaterTAP `Pump` model: the first uses the default value for the `variable_efficiency` argument (`"none"`) while the second uses the `"flow"` argument. The second build creates three additional variables (`bep_flow`, `bep_eta`, and `flow_ratio`) and two additional constraints (`flow_ratio_constraint`)

In [ ]:
eta = 0.8
bep_flow = 1e-3
salinity = 35  # g/L
flows = np.linspace(1e-4, bep_flow, 25)  # m3/s
temperature = 298  # K
inlet_pressure = 40 * pyunits.bar
deltaP = 10 * pyunits.bar

In [ ]:
##################################################################
# Build Pump without variable_efficiency

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()
# Add pump model to flowsheet
# Default value for variable_efficiency is "none"
m.fs.pump = Pump(property_package=m.fs.properties)

# Fix inlet conditions
m.fs.pump.control_volume.properties_in[0].flow_vol_phase["Liq"].fix(bep_flow)
m.fs.pump.control_volume.properties_in[0].conc_mass_phase_comp["Liq", "TDS"].fix(
    salinity
)
m.fs.pump.control_volume.properties_in[0].pressure.fix(inlet_pressure)
m.fs.pump.control_volume.properties_in[0].temperature.fix(temperature)

# Fix pump parameters
m.fs.pump.efficiency_pump.fix(eta)
m.fs.pump.deltaP.fix(deltaP)

# Solve model
assert degrees_of_freedom(m.fs.pump) == 0
results = solver.solve(m)
assert_optimal_termination(results)

# Create empty lists to store results
etas1 = []
power1 = []

for flow in flows:
    m.fs.pump.control_volume.properties_in[0].flow_vol_phase["Liq"].fix(flow)
    results = solver.solve(m)
    assert_optimal_termination(results)
    etas1.append(value(m.fs.pump.efficiency_pump[0]))
    power1.append(value(m.fs.pump.work_mechanical[0]))

##################################################################
# Build Pump with variable_efficiency

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()
# Add pump model to flowsheet
# Set variable_efficiency to "flow"
m.fs.pump = Pump(property_package=m.fs.properties, variable_efficiency="flow")

# Fix inlet conditions
m.fs.pump.control_volume.properties_in[0].flow_vol_phase["Liq"].fix(bep_flow)
m.fs.pump.control_volume.properties_in[0].conc_mass_phase_comp["Liq", "TDS"].fix(
    salinity
)
m.fs.pump.control_volume.properties_in[0].pressure.fix(inlet_pressure)
m.fs.pump.control_volume.properties_in[0].temperature.fix(temperature)

# Fix pump parameters
m.fs.pump.deltaP.fix(deltaP)
m.fs.pump.bep_eta.fix(eta)
m.fs.pump.bep_flow.fix(bep_flow)

# Solve model
assert degrees_of_freedom(m.fs.pump) == 0
results = solver.solve(m)
assert_optimal_termination(results)

# Create empty lists to store results
etas2 = []
power2 = []

for flow in flows:
    m.fs.pump.control_volume.properties_in[0].flow_vol_phase["Liq"].fix(flow)
    results = solver.solve(m)
    assert_optimal_termination(results)
    etas2.append(value(m.fs.pump.efficiency_pump[0]))
    power2.append(value(m.fs.pump.work_mechanical[0]))

##################################################################
# Plot results

fig, axs = plt.subplots(2, 1, figsize=(6, 7), sharex=True)
axs[0].plot(
    flows, etas1, marker="o", color="green", label="Without Variable Efficiency"
)
axs[1].plot(flows, power1, marker="o", color="green")
axs[0].plot(flows, etas2, marker="o", color="purple", label="With Variable Efficiency")
axs[1].plot(flows, power2, marker="o", color="purple")

axs[0].set_ylabel("Pump Efficiency")
axs[0].set_title("Pump Efficiency vs Flow Rate")
axs[1].set_xlabel("Flow rate (m3/s)")
axs[1].set_ylabel("Mechanical Power (W)")
axs[1].set_title("Pump Power vs Flow Rate")
axs[0].grid(visible=True)
axs[1].grid(visible=True)

axs[0].legend()
fig.tight_layout()

# RO Configuration Arguments

In [ ]:
pressures = np.linspace(35e5, 85e5, 10)

inlet_pressure = 50 * pyunits.bar
temperature = 298  # K
A_comp = 4.2e-12
B_comp = 3e-8
membrane_area = 50  # m2
flow = 1e-3  # m3/s
salinity = 35  # g/L
atmospheric = 101325  # Pa
deltaP = -3 * pyunits.bar
channel_height = 1 * pyunits.mm
spacer_porosity = 0.75
reflect_coeff = 0.95
cp_modulus = 1.1

# Flat Plate: Simple vs. Complex

In [ ]:
##################################################################
# FLAT SHEET - SD
# RO without concentration polarization, mass transfer coefficient, or pressure change

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="none",
    mass_transfer_coefficient="none",
    has_pressure_change=False,
    pressure_change_type="fixed_per_stage",
    transport_model="SD",
    module_type="flat_sheet",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)
m.fs.RO.permeate.pressure[0].fix(atmospheric)

m.fs.RO.area.fix(membrane_area)
m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)
calculate_scaling_factors(m)

m.fs.RO.initialize()

assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

recovery1 = []
perm_conc1 = []

for p in pressures:
    m.fs.RO.feed_side.properties_in[0].pressure.fix(p)
    results = solver.solve(m)
    assert_optimal_termination(results)
    recovery1.append(value(m.fs.RO.recovery_vol_phase[0, "Liq"]))
    perm_conc1.append(
        value(m.fs.RO.permeate_side[0, 0].conc_mass_phase_comp["Liq", "TDS"])
    )

##################################################################
# FLAT SHEET - SD
# RO with concentration polarization, mass transfer coefficient, and pressure change

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="calculated",
    mass_transfer_coefficient="calculated",
    has_pressure_change=True,
    pressure_change_type="calculated",
    transport_model="SD",
    friction_factor="default_by_module_type",
    module_type="flat_sheet",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)
m.fs.RO.permeate_side[0, 0].pressure.fix(atmospheric)

m.fs.RO.area.fix(membrane_area)
m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)
m.fs.RO.deltaP.fix(deltaP)
m.fs.RO.feed_side.channel_height.fix(channel_height)
m.fs.RO.feed_side.spacer_porosity.fix(spacer_porosity)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)

calculate_scaling_factors(m)

m.fs.RO.initialize()

assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

recovery2 = []
perm_conc2 = []

for p in pressures:
    m.fs.RO.feed_side.properties_in[0].pressure.fix(p)
    results = solver.solve(m)
    assert_optimal_termination(results)
    recovery2.append(value(m.fs.RO.recovery_vol_phase[0, "Liq"]))
    perm_conc2.append(
        value(m.fs.RO.permeate_side[0, 0].conc_mass_phase_comp["Liq", "TDS"])
    )

##################################################################
# Plot results

fig, ax = plt.subplots(2, 1, figsize=(6, 7), sharex=True)
ax[0].plot(
    pressures / 1e5,
    recovery1,
    marker="o",
    color="blue",
    label="Flat Sheet: Without CP, MTC, or ∆P",
)
ax[0].plot(
    pressures / 1e5,
    recovery2,
    marker="o",
    color="red",
    label="Flat Sheet: With CP, MTC, and ∆P",
)
ax[0].set_ylabel("Volumetric Recovery")
ax[0].set_title("RO Recovery vs Feed Pressure")
ax[0].grid(visible=True)
ax[0].legend()

ax[1].plot(
    pressures / 1e5,
    perm_conc1,
    marker="o",
    color="blue",
)
ax[1].plot(
    pressures / 1e5,
    perm_conc2,
    marker="o",
    color="red",
)
ax[1].set_xlabel("Feed Pressure (bar)")
ax[1].set_ylabel("TDS Concentration in Permeate (g/L)")
ax[1].set_title("Permeate Concentration vs Feed Pressure")
ax[1].grid(visible=True)
ax[0].legend()
fig.tight_layout()

# Spiral Wound: Simple vs. Complex

In [ ]:
##################################################################
# SPIRAL WOUND - SD
# RO without concentration polarization, mass transfer coefficient, or pressure change

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="none",
    mass_transfer_coefficient="none",
    has_pressure_change=False,
    pressure_change_type="fixed_per_stage",
    transport_model="SD",
    module_type="spiral_wound",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)
m.fs.RO.permeate.pressure[0].fix(atmospheric)

m.fs.RO.area.fix(membrane_area)
m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)

print(f"Degrees of freedom: {degrees_of_freedom(m)}")

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)
calculate_scaling_factors(m)

m.fs.RO.initialize()

assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

recovery1 = []
perm_conc1 = []

for p in pressures:
    m.fs.RO.feed_side.properties_in[0].pressure.fix(p)
    results = solver.solve(m)
    assert_optimal_termination(results)
    recovery1.append(value(m.fs.RO.recovery_vol_phase[0, "Liq"]))
    perm_conc1.append(
        value(m.fs.RO.permeate_side[0, 0].conc_mass_phase_comp["Liq", "TDS"])
    )


##################################################################
# SPIRAL WOUND - SD
# RO with concentration polarization, mass transfer coefficient, and pressure change

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="calculated",
    mass_transfer_coefficient="calculated",
    has_pressure_change=True,
    pressure_change_type="calculated",
    transport_model="SD",
    module_type="spiral_wound",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)
m.fs.RO.permeate_side[0, 0].pressure.fix(atmospheric)

m.fs.RO.area.fix(membrane_area)

m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)
m.fs.RO.deltaP.fix(deltaP)
m.fs.RO.feed_side.channel_height.fix(channel_height)
m.fs.RO.feed_side.spacer_porosity.fix(spacer_porosity)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)

calculate_scaling_factors(m)
m.fs.RO.initialize()

results = solver.solve(m)
assert_optimal_termination(results)

recovery2 = []
perm_conc2 = []

for p in pressures:
    m.fs.RO.feed_side.properties_in[0].pressure.fix(p)
    results = solver.solve(m)
    assert_optimal_termination(results)
    recovery2.append(value(m.fs.RO.recovery_vol_phase[0, "Liq"]))
    perm_conc2.append(
        value(m.fs.RO.permeate_side[0, 0].conc_mass_phase_comp["Liq", "TDS"])
    )

##################################################################
# Plot results

fig, ax = plt.subplots(2, 1, figsize=(6, 7), sharex=True)
ax[0].plot(
    pressures / 1e5,
    recovery1,
    marker="o",
    color="blue",
    label="Spiral Wound: Without CP, MTC, or ∆P",
)
ax[0].plot(
    pressures / 1e5,
    recovery2,
    marker="o",
    color="red",
    label="Spiral Wound: With CP, MTC, and ∆P",
)
ax[0].set_ylabel("Volumetric Recovery")
ax[0].set_title("RO Recovery vs Feed Pressure")
ax[0].grid(visible=True)
ax[0].legend()

ax[1].plot(
    pressures / 1e5,
    perm_conc1,
    marker="o",
    color="blue",
)
ax[1].plot(
    pressures / 1e5,
    perm_conc2,
    marker="o",
    color="red",
)
ax[1].set_xlabel("Feed Pressure (bar)")
ax[1].set_ylabel("TDS Concentration in Permeate (g/L)")
ax[1].set_title("Permeate Concentration vs Feed Pressure")
ax[1].grid(visible=True)

fig.tight_layout()

# Complex: Spiral Wound vs. Flat Sheet

In [ ]:
##################################################################
# FLAT SHEET - SD
# RO with concentration polarization, mass transfer coefficient, and pressure change

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="calculated",
    mass_transfer_coefficient="calculated",
    has_pressure_change=True,
    pressure_change_type="calculated",
    transport_model="SD",
    module_type="flat_sheet",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)
m.fs.RO.permeate.pressure[0].fix(atmospheric)

m.fs.RO.area.fix(membrane_area)
m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)
m.fs.RO.deltaP.fix(deltaP)
m.fs.RO.feed_side.channel_height.fix(channel_height)
m.fs.RO.feed_side.spacer_porosity.fix(spacer_porosity)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)

calculate_scaling_factors(m)
m.fs.RO.initialize()

assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

recovery1 = []
perm_conc1 = []

for p in pressures:
    m.fs.RO.feed_side.properties_in[0].pressure.fix(p)
    results = solver.solve(m)
    assert_optimal_termination(results)
    recovery1.append(value(m.fs.RO.recovery_vol_phase[0, "Liq"]))
    perm_conc1.append(
        value(m.fs.RO.permeate_side[0, 0].conc_mass_phase_comp["Liq", "TDS"])
    )

##################################################################
# SPIRAL WOUND - SD
# RO with concentration polarization, mass transfer coefficient, and pressure change

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="calculated",
    mass_transfer_coefficient="calculated",
    has_pressure_change=True,
    pressure_change_type="calculated",
    transport_model="SD",
    module_type="spiral_wound",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)
m.fs.RO.permeate_side[0, 0].pressure.fix(atmospheric)

m.fs.RO.area.fix(membrane_area)
m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)
m.fs.RO.deltaP.fix(deltaP)
m.fs.RO.feed_side.channel_height.fix(channel_height)
m.fs.RO.feed_side.spacer_porosity.fix(spacer_porosity)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)

calculate_scaling_factors(m)
m.fs.RO.initialize()

assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

recovery2 = []
perm_conc2 = []

for p in pressures:
    m.fs.RO.feed_side.properties_in[0].pressure.fix(p)
    results = solver.solve(m)
    assert_optimal_termination(results)
    recovery2.append(value(m.fs.RO.recovery_vol_phase[0, "Liq"]))
    perm_conc2.append(
        value(m.fs.RO.permeate_side[0, 0].conc_mass_phase_comp["Liq", "TDS"])
    )

#################################################################
# Plot results

fig, ax = plt.subplots(2, 1, figsize=(6, 7), sharex=True)
ax[0].plot(
    pressures / 1e5,
    recovery1,
    marker="o",
    color="blue",
    label="Flat Sheet",
)
ax[0].plot(
    pressures / 1e5,
    recovery2,
    marker="o",
    color="red",
    label="Spiral Wound",
)
ax[0].set_ylabel("Volumetric Recovery")
ax[0].set_title("RO Recovery vs Feed Pressure")
ax[0].grid(visible=True)
ax[0].legend()

ax[1].plot(
    pressures / 1e5,
    perm_conc1,
    marker="o",
    color="blue",
)
ax[1].plot(
    pressures / 1e5,
    perm_conc2,
    marker="o",
    color="red",
)
ax[1].set_xlabel("Feed Pressure (bar)")
ax[1].set_ylabel("TDS Concentration in Permeate (g/L)")
ax[1].set_title("Permeate Concentration vs Feed Pressure")
ax[1].grid(visible=True)

fig.tight_layout()

# Surface-Diffusion vs. Spiegler-Kedem-Katchalsky

In [ ]:
##################################################################
# SPIRAL WOUND - SD
# RO with concentration polarization, mass transfer coefficient, and pressure change

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="fixed",
    mass_transfer_coefficient="none",
    has_pressure_change=True,
    transport_model="SD",
    module_type="spiral_wound",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)
m.fs.RO.permeate.pressure[0].fix(atmospheric)

m.fs.RO.area.fix(membrane_area)
m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)
m.fs.RO.deltaP.fix(deltaP)
m.fs.RO.feed_side.cp_modulus.fix(cp_modulus)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)
calculate_scaling_factors(m)

m.fs.RO.initialize()

assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

recovery1 = []
perm_conc1 = []

for p in pressures:
    m.fs.RO.feed_side.properties_in[0].pressure.fix(p)
    results = solver.solve(m)
    assert_optimal_termination(results)
    recovery1.append(value(m.fs.RO.recovery_vol_phase[0, "Liq"]))
    perm_conc1.append(
        value(m.fs.RO.permeate_side[0, 0].conc_mass_phase_comp["Liq", "TDS"])
    )

##################################################################
# SPIRAL WOUND - SKK
# RO with concentration polarization, mass transfer coefficient, and pressure change

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type="fixed",
    mass_transfer_coefficient="none",
    has_pressure_change=True,
    transport_model="SKK",
    module_type="spiral_wound",
)

m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(0.035)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)
m.fs.RO.feed_side.properties_in[0].pressure.fix(inlet_pressure)
m.fs.RO.feed_side.properties_in[0].temperature.fix(temperature)
m.fs.RO.permeate_side[0, 0].pressure.fix(atmospheric)

m.fs.RO.reflect_coeff.fix(reflect_coeff)
m.fs.RO.area.fix(membrane_area)
m.fs.RO.A_comp.fix(A_comp)
m.fs.RO.B_comp.fix(B_comp)
m.fs.RO.deltaP.fix(deltaP)
m.fs.RO.feed_side.cp_modulus.fix(cp_modulus)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
set_scaling_factor(m.fs.RO.area, 1e-2)
calculate_scaling_factors(m)

m.fs.RO.initialize()

assert degrees_of_freedom(m) == 0
results = solver.solve(m)
assert_optimal_termination(results)

recovery2 = []
perm_conc2 = []

for p in pressures:
    m.fs.RO.feed_side.properties_in[0].pressure.fix(p)
    results = solver.solve(m)
    assert_optimal_termination(results)
    recovery2.append(value(m.fs.RO.recovery_vol_phase[0, "Liq"]))
    perm_conc2.append(
        value(m.fs.RO.permeate_side[0, 0].conc_mass_phase_comp["Liq", "TDS"])
    )

#################################################################
# Plot results

fig, ax = plt.subplots(2, 1, figsize=(6, 7), sharex=True)
ax[0].plot(
    pressures / 1e5,
    recovery1,
    marker="o",
    color="blue",
    label="Surface-Diffusion",
)
ax[0].plot(
    pressures / 1e5,
    recovery2,
    marker="o",
    color="red",
    label="Spiegler-Kedem-Katchalsky",
)
ax[0].set_ylabel("Volumetric Recovery")
ax[0].set_title("RO Recovery vs Feed Pressure")
ax[0].grid(visible=True)
ax[0].legend()

ax[1].plot(
    pressures / 1e5,
    perm_conc1,
    marker="o",
    color="blue",
)
ax[1].plot(
    pressures / 1e5,
    perm_conc2,
    marker="o",
    color="red",
)
ax[1].set_xlabel("Feed Pressure (bar)")
ax[1].set_ylabel("TDS Concentration in Permeate (g/L)")
ax[1].set_title("Permeate Concentration vs Feed Pressure")
ax[1].grid(visible=True)

fig.tight_layout()